# Pommerman Demo.

This notebook demonstrates how to train Pommerman agents. Please let us know at support@pommerman.com if you run into any issues.

In [3]:
import os
import sys
import numpy as np

from pommerman.agents import SimpleAgent, RandomAgent, PlayerAgent, BaseAgent
from pommerman.configs import ffa_v0_fast_env
from pommerman.envs.v0 import Pomme
from pommerman.characters import Bomber
from pommerman import utility

# Random agents

The following codes instantiates the environment with four random agents who take actions until the game is finished. (This will be a quick game.)

In [4]:
# Instantiate the environment
config = ffa_v0_fast_env()
env = Pomme(**config["env_kwargs"])

In [5]:
# Add four random agents
agents = {}
for agent_id in range(4):
    agents[agent_id] = RandomAgent(config["agent"](agent_id, config["game_type"]))
env.set_agents(list(agents.values()))
env.set_init_game_state(None)

In [6]:
# Seed and reset the environment
env.seed(0)
obs = env.reset()

# Run the random agents until we're done
done = False
while not done:
    env.render()
    actions = env.act(obs)
    obs, reward, done, info = env.step(actions)
env.render(close=True)
env.close()

print(info)

{'result': <Result.Win: 0>, 'winners': [3]}


# Human Agents

The following code runs the environment with 3 random agents and one agent with human input (use the arrow keys on your keyboard). This can also be called on the command line with:

`python run_battle.py --agents=player::arrows,random::null,random::null,random::null --config=PommeFFACompetition-v0`

You can also run this with SimpleAgents by executing:

`python run_battle.py --agents=player::arrows,test::agents.SimpleAgent,test::agents.SimpleAgent,test::agents.SimpleAgent --config=PommeFFACompetition-v0`

In [5]:
# Instantiate the environment
config = ffa_v0_fast_env()
env = Pomme(**config["env_kwargs"])

# Add 3 random agents
agents = {}
for agent_id in range(3):
    agents[agent_id] = RandomAgent(config["agent"](agent_id, config["game_type"]))

# Add human agent
agents[3] = PlayerAgent(config["agent"](agent_id, config["game_type"]), "arrows")

env.set_agents(list(agents.values()))
env.set_init_game_state(None)

In [6]:
# Seed and reset the environment
env.seed(0)
obs = env.reset()

# Run the agents until we're done
done = False
while not done:
    env.render()
    actions = env.act(obs)
    obs, reward, done, info = env.step(actions)
env.render(close=True)
env.close()

# Print the result
print(info)

{'result': <Result.Win: 0>, 'winners': [1]}


# Training an Agent

The following code uses Tensorforce to train a PPO agent. This is in the train_with_tensorforce.py module as well.

In [7]:
# Make sure you have tensorforce installed: pip install tensorforce
from tensorforce.agents import PPOAgent
from tensorforce.execution import Runner
from tensorforce.contrib.openai_gym import OpenAIGym

In [8]:
def make_np_float(feature):
    return np.array(feature).astype(np.float32)

def featurize(obs):
    board = obs["board"].reshape(-1).astype(np.float32)
    bomb_blast_strength = obs["bomb_blast_strength"].reshape(-1).astype(np.float32)
    bomb_life = obs["bomb_life"].reshape(-1).astype(np.float32)
    position = make_np_float(obs["position"])
    ammo = make_np_float([obs["ammo"]])
    blast_strength = make_np_float([obs["blast_strength"]])
    can_kick = make_np_float([obs["can_kick"]])

    teammate = obs["teammate"]
    if teammate is not None:
        teammate = teammate.value
    else:
        teammate = -1
    teammate = make_np_float([teammate])

    enemies = obs["enemies"]
    enemies = [e.value for e in enemies]
    if len(enemies) < 3:
        enemies = enemies + [-1]*(3 - len(enemies))
    enemies = make_np_float(enemies)

    return np.concatenate((board, bomb_blast_strength, bomb_life, position, ammo, blast_strength, can_kick, teammate, enemies))

class TensorforceAgent(BaseAgent):
    def act(self, obs, action_space):
        pass

In [9]:
config

{'agent': pommerman.characters.Bomber,
 'env_kwargs': {'game_type': <GameType.FFA: 1>,
  'board_size': 11,
  'num_rigid': 36,
  'num_wood': 36,
  'num_items': 20,
  'max_steps': 800,
  'render_fps': 1000,
  'env': 'pommerman.envs.v0:Pomme'},
 'env_id': 'PommeFFAFast-v0',
 'env_entry_point': 'pommerman.envs.v0:Pomme',
 'game_type': <GameType.FFA: 1>,
 'env': pommerman.envs.v0.Pomme}

In [43]:
# Instantiate the environment
config = ffa_v0_fast_env()
env = Pomme(**config["env_kwargs"])
env.seed(0)

# Create a Proximal Policy Optimization agent
agent = PPOAgent(
    states=dict(type='float', shape=env.observation_space.shape),
    actions=dict(type='int', num_actions=env.action_space.n),
    network=[
        dict(type='dense', size=64),
        dict(type='dense', size=64)
    ],
    batching_capacity=1000,
    step_optimizer=dict(
        type='adam',
        learning_rate=1e-4
    ),

    # PGModel
    baseline_mode='network',
    baseline=dict(type='custom', network=[
        dict(type='dense', size=64),
        dict(type='dense', size=64)
    ]),
    baseline_optimizer=dict(
        type='adam',
        learning_rate=1e-4
    ),
)

# Add 3 random agents
agents = []
for agent_id in range(3):
    agents.append(SimpleAgent(config["agent"](agent_id, config["game_type"])))

# Add TensorforceAgent
agent_id += 1
agents.append(TensorforceAgent(config["agent"](agent_id, config["game_type"])))
env.set_agents(agents)
env.set_training_agent(agents[-1].agent_id)
env.set_init_game_state(None)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [11]:
agents[-1]

In [12]:
env.reset()
len(env.get_observations())
for ob in env.get_observations():
    print('\n\n')
    print(ob)
    print('\n\n')




{'alive': [10, 11, 12, 13], 'board': array([[ 0,  1,  0,  0,  1,  0,  1,  2,  1,  1,  0],
       [ 1, 10,  0,  0,  2,  2,  2,  0,  0, 13,  2],
       [ 0,  0,  0,  2,  0,  2,  1,  2,  1,  0,  1],
       [ 0,  0,  2,  0,  2,  1,  1,  1,  1,  0,  0],
       [ 1,  2,  0,  2,  0,  0,  0,  1,  1,  2,  2],
       [ 0,  2,  2,  1,  0,  0,  2,  1,  0,  2,  2],
       [ 1,  2,  1,  1,  0,  2,  0,  2,  0,  2,  2],
       [ 2,  0,  2,  1,  1,  1,  2,  0,  1,  0,  1],
       [ 1,  0,  1,  1,  1,  0,  0,  1,  0,  0,  2],
       [ 1, 11,  0,  0,  2,  2,  2,  0,  0, 12,  1],
       [ 0,  2,  1,  0,  2,  2,  2,  1,  2,  1,  0]], dtype=uint8), 'bomb_blast_strength': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0

In [13]:
class WrappedEnv(OpenAIGym):    
    def __init__(self, gym, visualize=False):
        self.gym = gym
        self.visualize = visualize
    
    def execute(self, action):
        if self.visualize:
            self.gym.render()

        actions = self.unflatten_action(action=action)

        obs = self.gym.get_observations()
        all_actions = self.gym.act(obs)
        all_actions.insert(self.gym.training_agent, actions)
        state, reward, terminal, _ = self.gym.step(all_actions)
        agent_state = featurize(state[self.gym.training_agent])
        agent_reward = reward[self.gym.training_agent]
        return agent_state, terminal, agent_reward
    
    def reset(self):
        obs = self.gym.reset()
        agent_obs = featurize(obs[3])
        return agent_obs

In [44]:
# Instantiate and run the environment for 5 episodes.

# Callback function printing episode statistics
def episode_finished(r):
    if r.episode % 100 == 0:
        print("Finished episode {ep} after {ts} timesteps (reward: {reward})".format(ep=r.episode, ts=r.episode_timestep,
                                                                                     reward=r.episode_rewards[-1]))
    return True


wrapped_env = WrappedEnv(env, visualize=False)
runner = Runner(agent=agent, environment=wrapped_env)
runner.run(episodes=120000, max_episode_timesteps=2000, episode_finished=episode_finished)
# print("Stats: ", runner.episode_rewards, runner.episode_timesteps, runner.episode_times)

agent.save_model('/Users/voiceup/Git/cs221-pommerman/playground/notebooks/saved_ckpts/')

try:
    runner.close()
except AttributeError as e:
    pass

Finished episode 100 after 24 timesteps (reward: -1)
Finished episode 200 after 87 timesteps (reward: -1)
Finished episode 300 after 58 timesteps (reward: -1)
Finished episode 400 after 111 timesteps (reward: -1)
Finished episode 500 after 158 timesteps (reward: -1)
Finished episode 600 after 269 timesteps (reward: -1)
Finished episode 700 after 69 timesteps (reward: -1)
Finished episode 800 after 101 timesteps (reward: -1)
Finished episode 900 after 360 timesteps (reward: -1)
Finished episode 1000 after 416 timesteps (reward: -1)
Finished episode 1100 after 126 timesteps (reward: -1)
Finished episode 1200 after 124 timesteps (reward: 1)
Finished episode 1300 after 296 timesteps (reward: -1)
Finished episode 1400 after 165 timesteps (reward: -1)
Finished episode 1500 after 135 timesteps (reward: -1)
Finished episode 1600 after 254 timesteps (reward: -1)
Finished episode 1700 after 132 timesteps (reward: -1)
Finished episode 1800 after 240 timesteps (reward: -1)
Finished episode 1900 af

Finished episode 15000 after 118 timesteps (reward: -1)
Finished episode 15100 after 127 timesteps (reward: -1)
Finished episode 15200 after 142 timesteps (reward: -1)
Finished episode 15300 after 333 timesteps (reward: -1)
Finished episode 15400 after 242 timesteps (reward: -1)
Finished episode 15500 after 84 timesteps (reward: -1)
Finished episode 15600 after 113 timesteps (reward: -1)
Finished episode 15700 after 129 timesteps (reward: -1)
Finished episode 15800 after 140 timesteps (reward: -1)
Finished episode 15900 after 132 timesteps (reward: -1)
Finished episode 16000 after 114 timesteps (reward: -1)
Finished episode 16100 after 175 timesteps (reward: -1)
Finished episode 16200 after 97 timesteps (reward: -1)
Finished episode 16300 after 144 timesteps (reward: -1)
Finished episode 16400 after 801 timesteps (reward: -1)
Finished episode 16500 after 334 timesteps (reward: 1)
Finished episode 16600 after 151 timesteps (reward: -1)
Finished episode 16700 after 110 timesteps (reward:

Finished episode 29700 after 115 timesteps (reward: -1)
Finished episode 29800 after 201 timesteps (reward: -1)
Finished episode 29900 after 192 timesteps (reward: -1)
Finished episode 30000 after 231 timesteps (reward: -1)
Finished episode 30100 after 122 timesteps (reward: -1)
Finished episode 30200 after 147 timesteps (reward: -1)
Finished episode 30300 after 142 timesteps (reward: -1)
Finished episode 30400 after 168 timesteps (reward: -1)
Finished episode 30500 after 187 timesteps (reward: -1)
Finished episode 30600 after 135 timesteps (reward: -1)
Finished episode 30700 after 173 timesteps (reward: -1)
Finished episode 30800 after 88 timesteps (reward: -1)
Finished episode 30900 after 243 timesteps (reward: -1)
Finished episode 31000 after 152 timesteps (reward: -1)
Finished episode 31100 after 116 timesteps (reward: 1)
Finished episode 31200 after 86 timesteps (reward: -1)
Finished episode 31300 after 168 timesteps (reward: -1)
Finished episode 31400 after 296 timesteps (reward:

Finished episode 44400 after 141 timesteps (reward: -1)
Finished episode 44500 after 98 timesteps (reward: -1)
Finished episode 44600 after 133 timesteps (reward: -1)
Finished episode 44700 after 166 timesteps (reward: -1)
Finished episode 44800 after 102 timesteps (reward: -1)
Finished episode 44900 after 91 timesteps (reward: -1)
Finished episode 45000 after 91 timesteps (reward: -1)
Finished episode 45100 after 123 timesteps (reward: -1)
Finished episode 45200 after 236 timesteps (reward: -1)
Finished episode 45300 after 322 timesteps (reward: -1)
Finished episode 45400 after 170 timesteps (reward: -1)
Finished episode 45500 after 416 timesteps (reward: -1)
Finished episode 45600 after 467 timesteps (reward: -1)
Finished episode 45700 after 193 timesteps (reward: -1)
Finished episode 45800 after 104 timesteps (reward: -1)
Finished episode 45900 after 85 timesteps (reward: -1)
Finished episode 46000 after 112 timesteps (reward: 1)
Finished episode 46100 after 121 timesteps (reward: -

Finished episode 59100 after 435 timesteps (reward: -1)
Finished episode 59200 after 172 timesteps (reward: 1)
Finished episode 59300 after 120 timesteps (reward: -1)
Finished episode 59400 after 148 timesteps (reward: -1)
Finished episode 59500 after 105 timesteps (reward: -1)
Finished episode 59600 after 150 timesteps (reward: -1)
Finished episode 59700 after 190 timesteps (reward: -1)
Finished episode 59800 after 344 timesteps (reward: -1)
Finished episode 59900 after 141 timesteps (reward: -1)
Finished episode 60000 after 185 timesteps (reward: -1)
Finished episode 60100 after 452 timesteps (reward: -1)
Finished episode 60200 after 364 timesteps (reward: -1)
Finished episode 60300 after 319 timesteps (reward: -1)
Finished episode 60400 after 112 timesteps (reward: -1)
Finished episode 60500 after 252 timesteps (reward: -1)
Finished episode 60600 after 135 timesteps (reward: -1)
Finished episode 60700 after 119 timesteps (reward: -1)
Finished episode 60800 after 73 timesteps (reward

Finished episode 73800 after 101 timesteps (reward: -1)
Finished episode 73900 after 90 timesteps (reward: -1)
Finished episode 74000 after 108 timesteps (reward: -1)
Finished episode 74100 after 160 timesteps (reward: -1)
Finished episode 74200 after 299 timesteps (reward: 1)
Finished episode 74300 after 81 timesteps (reward: -1)
Finished episode 74400 after 79 timesteps (reward: -1)
Finished episode 74500 after 226 timesteps (reward: -1)
Finished episode 74600 after 220 timesteps (reward: -1)
Finished episode 74700 after 177 timesteps (reward: -1)
Finished episode 74800 after 229 timesteps (reward: -1)
Finished episode 74900 after 317 timesteps (reward: -1)
Finished episode 75000 after 142 timesteps (reward: -1)
Finished episode 75100 after 254 timesteps (reward: -1)
Finished episode 75200 after 217 timesteps (reward: -1)
Finished episode 75300 after 158 timesteps (reward: -1)
Finished episode 75400 after 88 timesteps (reward: -1)
Finished episode 75500 after 311 timesteps (reward: -

Finished episode 88500 after 559 timesteps (reward: -1)
Finished episode 88600 after 138 timesteps (reward: -1)
Finished episode 88700 after 379 timesteps (reward: -1)
Finished episode 88800 after 92 timesteps (reward: -1)
Finished episode 88900 after 364 timesteps (reward: 1)
Finished episode 89000 after 155 timesteps (reward: 1)
Finished episode 89100 after 140 timesteps (reward: -1)
Finished episode 89200 after 581 timesteps (reward: -1)
Finished episode 89300 after 132 timesteps (reward: -1)
Finished episode 89400 after 801 timesteps (reward: -1)
Finished episode 89500 after 84 timesteps (reward: -1)
Finished episode 89600 after 79 timesteps (reward: -1)
Finished episode 89700 after 242 timesteps (reward: -1)
Finished episode 89800 after 192 timesteps (reward: -1)
Finished episode 89900 after 128 timesteps (reward: -1)
Finished episode 90000 after 181 timesteps (reward: -1)
Finished episode 90100 after 121 timesteps (reward: -1)
Finished episode 90200 after 227 timesteps (reward: -

Finished episode 103200 after 106 timesteps (reward: -1)
Finished episode 103300 after 67 timesteps (reward: -1)
Finished episode 103400 after 200 timesteps (reward: 1)
Finished episode 103500 after 128 timesteps (reward: -1)
Finished episode 103600 after 278 timesteps (reward: -1)
Finished episode 103700 after 130 timesteps (reward: -1)
Finished episode 103800 after 189 timesteps (reward: 1)
Finished episode 103900 after 110 timesteps (reward: -1)
Finished episode 104000 after 322 timesteps (reward: -1)
Finished episode 104100 after 166 timesteps (reward: -1)
Finished episode 104200 after 92 timesteps (reward: -1)
Finished episode 104300 after 120 timesteps (reward: -1)
Finished episode 104400 after 644 timesteps (reward: -1)
Finished episode 104500 after 128 timesteps (reward: -1)
Finished episode 104600 after 187 timesteps (reward: -1)
Finished episode 104700 after 374 timesteps (reward: -1)
Finished episode 104800 after 263 timesteps (reward: -1)
Finished episode 104900 after 249 t

Finished episode 117700 after 163 timesteps (reward: -1)
Finished episode 117800 after 243 timesteps (reward: -1)
Finished episode 117900 after 110 timesteps (reward: -1)
Finished episode 118000 after 66 timesteps (reward: 1)
Finished episode 118100 after 96 timesteps (reward: -1)
Finished episode 118200 after 146 timesteps (reward: -1)
Finished episode 118300 after 528 timesteps (reward: -1)
Finished episode 118400 after 520 timesteps (reward: -1)
Finished episode 118500 after 235 timesteps (reward: -1)
Finished episode 118600 after 122 timesteps (reward: -1)
Finished episode 118700 after 148 timesteps (reward: -1)
Finished episode 118800 after 147 timesteps (reward: -1)
Finished episode 118900 after 165 timesteps (reward: 1)
Finished episode 119000 after 144 timesteps (reward: -1)
Finished episode 119100 after 127 timesteps (reward: -1)
Finished episode 119200 after 119 timesteps (reward: 1)
Finished episode 119300 after 176 timesteps (reward: -1)
Finished episode 119400 after 111 ti

In [36]:
import os

os.path.abspath(path='.')

'/Users/voiceup/Git/cs221-pommerman/playground/notebooks'

# TensorForce Tutorial

In [17]:
import numpy as np

from tensorforce.agents import PPOAgent
from tensorforce.execution import Runner
from tensorforce.contrib.openai_gym import OpenAIGym

# Create an OpenAIgym environment
env = OpenAIGym('CartPole-v0', visualize=True)

# Network as list of layers
network_spec = [
    dict(type='dense', size=32, activation='tanh'),
    dict(type='dense', size=32, activation='tanh')
]

agent = PPOAgent(
    states=env.states,
    actions=env.actions,
    network=network_spec,
#     batch_size=4096,
    # PPOAgent
    step_optimizer=dict(
        type='adam',
        learning_rate=1e-3
    ),
#     # Model
#     discount=0.99,
#     # DistributionModel
#     # PGModel
#     baseline_mode=None,
#     baseline=None,
#     baseline_optimizer=None,
#     gae_lambda=None,
#     # PGLRModel
#     likelihood_ratio_clipping=0.2,
)

# Create the runner
runner = Runner(agent=agent, environment=env)


# Callback function printing episode statistics
def episode_finished(r):
    print("Finished episode {ep} after {ts} timesteps (reward: {reward})".format(ep=r.episode, ts=r.episode_timestep,
                                                                                 reward=r.episode_rewards[-1]))
    return True


# Start learning
runner.run(episodes=3000, max_episode_timesteps=200, episode_finished=episode_finished)
runner.close()

# Print statistics
print("Learning finished. Total episodes: {ep}. Average reward of last 100 episodes: {ar}.".format(
    ep=runner.episode,
    ar=np.mean(runner.episode_rewards[-100:]))
)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Finished episode 1 after 44 timesteps (reward: 44.0)
Finished episode 2 after 19 timesteps (reward: 19.0)
Finished episode 3 after 10 timesteps (reward: 10.0)
Finished episode 4 after 9 timesteps (reward: 9.0)
Finished episode 5 after 27 timesteps (reward: 27.0)
Finished episode 6 after 35 timesteps (reward: 35.0)
Finished episode 7 after 23 timesteps (reward: 23.0)
Finished episode 8 after 10 timesteps (reward: 10.0)
Finished episode 9 after 22 timesteps (reward: 22.0)
Finished episode 10 after 11 timesteps (reward: 11.0)
Finished episode 11 after 25 timesteps (reward: 25.0)
Finished episode 12 after 28 timesteps (reward: 28.0)
Finished episode 13 after 20 timesteps (reward: 20.0)
Finished episode 14 after 77 timesteps (reward: 77.0)
Finished episode 15 after 24 timesteps (reward: 24.0)
Finished episode 16 after 18 timesteps (reward: 18.0)
Finished episode 17 after 2

Finished episode 146 after 160 timesteps (reward: 160.0)
Finished episode 147 after 200 timesteps (reward: 200.0)
Finished episode 148 after 200 timesteps (reward: 200.0)
Finished episode 149 after 200 timesteps (reward: 200.0)
Finished episode 150 after 172 timesteps (reward: 172.0)
Finished episode 151 after 181 timesteps (reward: 181.0)
Finished episode 152 after 172 timesteps (reward: 172.0)
Finished episode 153 after 166 timesteps (reward: 166.0)
Finished episode 154 after 200 timesteps (reward: 200.0)
Finished episode 155 after 179 timesteps (reward: 179.0)
Finished episode 156 after 158 timesteps (reward: 158.0)
Finished episode 157 after 200 timesteps (reward: 200.0)
Finished episode 158 after 120 timesteps (reward: 120.0)
Finished episode 159 after 200 timesteps (reward: 200.0)
Finished episode 160 after 200 timesteps (reward: 200.0)
Finished episode 161 after 200 timesteps (reward: 200.0)
Finished episode 162 after 182 timesteps (reward: 182.0)
Finished episode 163 after 200 

Finished episode 290 after 200 timesteps (reward: 200.0)
Finished episode 291 after 200 timesteps (reward: 200.0)
Finished episode 292 after 200 timesteps (reward: 200.0)
Finished episode 293 after 200 timesteps (reward: 200.0)
Finished episode 294 after 179 timesteps (reward: 179.0)
Finished episode 295 after 200 timesteps (reward: 200.0)
Finished episode 296 after 200 timesteps (reward: 200.0)
Finished episode 297 after 200 timesteps (reward: 200.0)
Finished episode 298 after 200 timesteps (reward: 200.0)
Finished episode 299 after 182 timesteps (reward: 182.0)
Finished episode 300 after 200 timesteps (reward: 200.0)
Finished episode 301 after 200 timesteps (reward: 200.0)
Finished episode 302 after 200 timesteps (reward: 200.0)
Finished episode 303 after 200 timesteps (reward: 200.0)
Finished episode 304 after 200 timesteps (reward: 200.0)
Finished episode 305 after 149 timesteps (reward: 149.0)
Finished episode 306 after 200 timesteps (reward: 200.0)
Finished episode 307 after 200 

Finished episode 434 after 200 timesteps (reward: 200.0)
Finished episode 435 after 200 timesteps (reward: 200.0)
Finished episode 436 after 200 timesteps (reward: 200.0)
Finished episode 437 after 200 timesteps (reward: 200.0)
Finished episode 438 after 200 timesteps (reward: 200.0)
Finished episode 439 after 200 timesteps (reward: 200.0)
Finished episode 440 after 200 timesteps (reward: 200.0)
Finished episode 441 after 200 timesteps (reward: 200.0)
Finished episode 442 after 200 timesteps (reward: 200.0)
Finished episode 443 after 200 timesteps (reward: 200.0)
Finished episode 444 after 200 timesteps (reward: 200.0)
Finished episode 445 after 200 timesteps (reward: 200.0)
Finished episode 446 after 200 timesteps (reward: 200.0)
Finished episode 447 after 200 timesteps (reward: 200.0)
Finished episode 448 after 184 timesteps (reward: 184.0)
Finished episode 449 after 200 timesteps (reward: 200.0)
Finished episode 450 after 200 timesteps (reward: 200.0)
Finished episode 451 after 200 

Finished episode 578 after 200 timesteps (reward: 200.0)
Finished episode 579 after 200 timesteps (reward: 200.0)
Finished episode 580 after 200 timesteps (reward: 200.0)
Finished episode 581 after 200 timesteps (reward: 200.0)
Finished episode 582 after 200 timesteps (reward: 200.0)
Finished episode 583 after 200 timesteps (reward: 200.0)
Finished episode 584 after 200 timesteps (reward: 200.0)
Finished episode 585 after 200 timesteps (reward: 200.0)
Finished episode 586 after 200 timesteps (reward: 200.0)
Finished episode 587 after 200 timesteps (reward: 200.0)
Finished episode 588 after 200 timesteps (reward: 200.0)
Finished episode 589 after 200 timesteps (reward: 200.0)
Finished episode 590 after 200 timesteps (reward: 200.0)
Finished episode 591 after 200 timesteps (reward: 200.0)
Finished episode 592 after 200 timesteps (reward: 200.0)
Finished episode 593 after 200 timesteps (reward: 200.0)
Finished episode 594 after 200 timesteps (reward: 200.0)
Finished episode 595 after 200 

KeyboardInterrupt: 